In [1]:
from LarpixParser import event_parser as EvtParser
from LarpixParser import hit_parser as HitParser
from LarpixParser import geom_dict_loader as DictLoader
from LarpixParser import util as util

import h5py
import numpy as np
import plotly.graph_objects as go
import yaml

In [17]:
def draw_cathode_planes(geom_dict, run_config_path, **kwargs):

    with open(run_config_path) as infile:
        run_yaml = yaml.load(infile, Loader=yaml.FullLoader)

    tpc_centers = run_yaml['tpc_offsets'] #cm
    
    pos = np.array(list(geom_dict.values())) / 10.
    traces = []
    
    for i_mod in range(len(tpc_centers)):
        x = np.array([pos[:, 0].min() + tpc_centers[i_mod][2], pos[:, 0].max() + tpc_centers[i_mod][2]])
        y = np.array([pos[:, 1].min() + tpc_centers[i_mod][1], pos[:, 1].max() + tpc_centers[i_mod][1]])
        z = np.full((2,2),  tpc_centers[i_mod][0])
        

        traces.append(go.Surface(x=x, y=y, z=z, **kwargs))
    
    return traces

In [18]:
def draw_anode_planes(geom_dict, run_config_path, **kwargs):

    with open(run_config_path) as infile:
        run_yaml = yaml.load(infile, Loader=yaml.FullLoader)

    tpc_centers = run_yaml['tpc_offsets'] #cm
    
    pos = np.array(list(geom_dict.values())) / 10.
    traces = []
    
    for i_mod in range(len(tpc_centers)):
        x = np.array([pos[:, 0].min() + tpc_centers[i_mod][2], pos[:, 0].max() + tpc_centers[i_mod][2]])
        y = np.array([pos[:, 1].min() + tpc_centers[i_mod][1], pos[:, 1].max() + tpc_centers[i_mod][1]])
        z1 = np.full((2,2),  pos[:, 2].min() + tpc_centers[i_mod][0])
        z2 = np.full((2,2),  pos[:, 2].max() + tpc_centers[i_mod][0])
        

        traces.append(go.Surface(x=x, y=y, z=z1, **kwargs))
        traces.append(go.Surface(x=x, y=y, z=z2, **kwargs))
    
    return traces

In [3]:
f = h5py.File('example_data/ndlar_ev20_larndsim.h5', 'r')
packets = f['packets'] # readout
G4_segments = f['tracks'] # Geant4 truth
assn = f['mc_packets_assn'] # G4-readout association

In [4]:
geom_dict = DictLoader.load_geom_dict("config_repo/dict_repo/multi_tile_layout-3.0.40.pkl")

run_config_path = 'config_repo/ndlar-module.yaml'
run_config = util.get_run_config(run_config_path)

In [5]:
with open(run_config_path) as infile:
    run_yaml = yaml.load(infile, Loader=yaml.FullLoader)

tpc_centers = run_yaml['tpc_offsets'] #cm

In [6]:
pckt_event_ids = EvtParser.packet_to_eventid(assn, G4_segments)
event_ids = np.unique(pckt_event_ids[pckt_event_ids != -1]) 
t0_grp = EvtParser.get_t0(packets)

In [7]:
switch_xz = False

In [25]:
i_ev = 0

pckt_mask = pckt_event_ids == event_ids[i_ev]
packets_ev = packets[pckt_mask]
assn_ev = assn[pckt_mask]
t0 = t0_grp[i_ev][0]

x,y,z,dQ = HitParser.hit_parser_charge(t0, packets_ev, geom_dict, run_config, switch_xz)

x = np.array(x) / 10.
y = np.array(y) / 10.
z = np.array(z) / 10.


In [29]:
fig = go.Figure()

fig.add_traces(draw_cathode_planes(
    geom_dict,
    run_config_path,
    showscale=False,
    opacity=0.3,
    colorscale='Greys',
))

fig.add_traces(draw_anode_planes(
    geom_dict,
    run_config_path,
    showscale=False,
    opacity=0.1,
    colorscale='ice',
))

trace = go.Scatter3d(
        x=x, y=y, z=z,
        marker_color=dQ,
        name='Reco3D',
        mode='markers',
        marker_size=3,
        marker_symbol='square',
        opacity=1.
        )

fig.update_layout(
    width=1024, height=768,
    legend_orientation="v",
)

fig.add_traces(trace)
fig.show()